In [2]:
import random
import numpy as np
import igraph
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
import nltk
import csv

nltk.download('punkt') # for tokenization
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/vpro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/vpro/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]
#print (testing_set)

In [6]:
###################
# random baseline #
###################

random_predictions = np.random.choice([0, 1], size=len(testing_set))
random_predictions = zip(range(len(testing_set)),random_predictions)

with open("random_predictions.csv","w") as pred:
    csv_out = csv.writer(pred)
    for row in random_predictions:
        csv_out.writerow(row)
        

In [4]:
# note: Kaggle requires that you add "ID" and "category" column headers

###############################
# beating the random baseline #
###############################

# the following script gets an F1 score of approximately 0.66

# data loading and preprocessing 

# the columns of the data frame below are: 
# (1) paper unique ID (integer)
# (2) publication year (integer)
# (3) paper title (string)
# (4) authors (strings separated by ,)
# (5) name of journal (optional) (string)
# (6) abstract (string) - lowercased, free of punctuation except intra-word dashes

with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]


In [5]:
with open("node_information.csv", "r") as f:
    reader = csv.reader(f, delimiter=';')
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

print (len(IDs))

27768


In [81]:
np.array(node_info).shape

(27768, 6)

In [92]:
## the following shows how to construct a graph with igraph
## even though in this baseline we don't use it
## look at http://igraph.org/python/doc/igraph.Graph-class.html for feature ideas

#edges = [(element[0],element[1]) for element in training_set if element[2]=="1"]

## some nodes may not be connected to any other node
## hence the need to create the nodes of the graph from node_info.csv,
## not just from the edge list

#nodes = IDs

## create empty directed graph
#g = igraph.Graph(directed=True)
 
## add vertices
#g.add_vertices(nodes)
 
## add edges
#g.add_edges(edges)

# for each training example we need to compute features
# in this baseline we will train the model on only 5% of the training set




In [6]:
# randomly select 5% of training set
to_keep = random.sample(range(len(training_set)), k=int(round(len(training_set)*0.05)))
training_set_reduced = [training_set[i] for i in to_keep]

In [7]:
abstract_corpus = [element[5].split(' ') for element in node_info]

In [8]:
import gensim, logging
model = gensim.models.Word2Vec(abstract_corpus,size=200, min_count=1)
    

In [234]:
model.sg

0

In [ ]:
a =cosine_similarity ([ model.wv[element] for element in abstract_corpus[3][np.random.uniform(0,len(abstract_corpus[3]),10).tolist()] ], [ model.wv[element] for element in abstract_corpus[1] ])

In [241]:
print(a.shape)
np.mean(a)

(109, 172)


0.080316342

In [157]:
vec1 = 
cosine_similarity([[0,2,3], [11,5,3]], [[0,2,0], [1,5,3]])

array([[ 0.5547002 ,  0.89073374],
       [ 0.40160966,  0.61095981]])

In [25]:
l =["hhh", "jjj"]
print (l+["h"])

['hhh', 'jjj', 'h']


In [9]:
def regulate_list_size(l, size = 25):
    if len(l)==25:
        return l
    if len(l)>size:
        return l[:25]
    else:
        diff = 25-len(l)
        return l+["" for _ in range(diff)]

In [34]:
len(regulate_list_size(["hhh" for _ in range(26)]))

25

In [10]:
def average_sentences_distance(model, sentence_one, sentence_two):
    S1 = regulate_list_size(sentence_one)
    S2 = regulate_list_size(sentence_two)
    a =cosine_similarity ([ model.wv[element] for element in S1 ], [ model.wv[element] for element in S2 ])
    return a.flatten() #np.mean(a)
    

In [44]:
average_sentences_distance(model, abstract_corpus[10], abstract_corpus[1]).shape

(625,)

In [145]:
# import modules & set up logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = [['nawfal' ,'is', 'bad'], ['yassir' ,'is' ,'good'], ['yassir' ,'is' ,'very','good']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)
#print(model.wv['nawfal'])
model.most_similar('good', topn=5)

[('yassir', 0.07428740710020065),
 ('is', -0.00040498003363609314),
 ('very', -0.04391438513994217),
 ('bad', -0.05098734796047211),
 ('nawfal', -0.08526964485645294)]

In [ ]:
def build_corpus(citations_set):
    #print(training_set)
    # compute TFIDF vector of each paper
    #corpus = []
    #for index,element in enumerate(node_info):
    #    row = element[0].split(';')
    #    print(row[len(row)-1])
        #print(np.array(row).shape)
        #print (len(row))
        #if len(row) == 6:
        #    corpus.append(row[5])
        #else:
        #    print(index)
        #    corpus.append(row[2])

    #corpus = [element[5] for element in node_info]
    #vectorizer = TfidfVectorizer(stop_words="english")

    # each row is a node in the order of node_info
    #features_TFIDF = vectorizer.fit_transform(corpus)

    #print (corpus[0])

In [116]:
################################################################
# Building the autor graph for distance between autors feature #
################################################################

#################### Feature description #######################
# at each time a couple (author_source, author_destination)    #
# appear the distance between their two nodes should decrease  #
# at prediction time we should reculculate this graph for the  # 
# evaluation set                                               #
################################################################

def build_authors_graph(citations_set):
    authors_nodes= []
    authors_edges = []

    counter = 0
    for i in range(len(citations_set)):

        source = citations_set[i][0]
        target = citations_set[i][1]

        try:
            source_info = [element for element in node_info if element[0]==source][0]
        except:
            source_info = [-1,-1,"","","",""] #[-1,-1,"nan","nan","nan","nan"] #["nan"]*6

        try:
            target_info = [element for element in node_info if element[0]==target][0]
        except:
            target_info = [-1,-1,"","","",""] #["nan"]*6

        source_auth = source_info[3].split(",")
        target_auth = target_info[3].split(",")

        for auth_source in source_auth:
            if auth_source not in authors_nodes:
                authors_nodes.append(auth_source)
            for auth_target in target_auth:
                if auth_target not in authors_nodes:
                    authors_nodes.append(auth_target)
                authors_edges.append(([auth_source, auth_target]))
        counter += 1
        if counter % 1000 == True:
            print(counter, "citation processsed")
    
    ## create empty directed graph
    g_authors = igraph.Graph(directed=True)

    ## add vertices
    g_authors.add_vertices(authors_nodes)

    ## add edges
    g_authors.add_edges(authors_edges)
    
    return g_authors
    

In [ ]:
## create empty directed graph
g = igraph.Graph(directed=True)
nodes = [1,2,5]
edges = [([1,2]), ([2,5])]
## add vertices
g.add_vertices(nodes)

## add edges
g.add_edges(edges)

In [122]:
def authors_distance(graph, source_list, target_list):
    distance = 0
    for source in source_list:
        for target in target_list:
            print(graph.shortest_paths_dijkstra(source, target))


In [126]:
#g_authors.es
for e in g_authors.es:
    print (e.tuple)


(0, 1)
(2, 1)
(3, 4)
(3, 5)
(6, 4)
(6, 5)
(7, 4)
(7, 5)
(8, 9)
(10, 11)
(12, 11)
(13, 14)
(13, 15)
(8, 16)
(8, 17)
(8, 18)
(8, 19)
(20, 21)
(20, 22)
(23, 21)
(23, 22)
(8, 8)
(24, 25)
(24, 26)
(24, 27)
(28, 25)
(28, 26)
(28, 27)
(29, 30)
(29, 31)
(32, 30)
(32, 31)
(33, 30)
(33, 31)
(34, 35)
(34, 31)
(36, 35)
(36, 31)
(37, 35)
(37, 31)
(38, 39)
(38, 40)
(41, 39)
(41, 40)
(42, 39)
(42, 40)
(43, 39)
(43, 40)
(44, 8)
(45, 8)
(46, 47)
(46, 48)
(49, 47)
(49, 48)
(50, 47)
(50, 48)
(51, 47)
(51, 48)
(52, 53)
(54, 53)
(55, 53)
(56, 57)
(56, 58)
(59, 57)
(59, 58)
(60, 61)
(60, 62)
(60, 63)
(64, 61)
(64, 62)
(64, 63)
(37, 61)
(37, 62)
(37, 63)
(65, 61)
(65, 62)
(65, 63)
(66, 61)
(66, 62)
(66, 63)
(67, 61)
(67, 62)
(67, 63)
(68, 34)
(68, 69)
(70, 34)
(70, 69)
(71, 72)
(71, 73)
(74, 75)
(74, 76)
(74, 77)
(78, 75)
(78, 76)
(78, 77)
(79, 80)
(79, 81)
(79, 82)
(83, 80)
(83, 81)
(83, 82)
(84, 8)
(85, 8)
(86, 8)
(87, 88)
(87, 89)
(87, 90)
(87, 91)
(92, 88)
(92, 89)
(92, 90)
(92, 91)
(93, 88)
(93, 89)
(93

(632, 631)
(8, 633)
(8, 634)
(8, 635)
(8, 514)
(8, 636)
(8, 284)
(637, 8)
(8, 638)
(8, 639)
(8, 640)
(8, 641)
(642, 643)
(642, 644)
(645, 643)
(645, 644)
(646, 643)
(646, 644)
(647, 8)
(648, 649)
(648, 650)
(651, 652)
(651, 653)
(651, 654)
(651, 566)
(651, 567)
(651, 655)
(656, 657)
(658, 657)
(27, 657)
(659, 657)
(164, 660)
(164, 661)
(164, 27)
(164, 662)
(167, 660)
(167, 661)
(167, 27)
(167, 662)
(168, 660)
(168, 661)
(168, 27)
(168, 662)
(663, 615)
(663, 664)
(665, 8)
(666, 667)
(666, 668)
(499, 667)
(499, 668)
(73, 667)
(73, 668)
(669, 670)
(669, 671)
(672, 673)
(672, 674)
(672, 142)
(672, 675)
(672, 676)
(8, 677)
(8, 678)
(8, 679)
(680, 681)
(680, 682)
(680, 241)
(680, 683)
(684, 681)
(684, 682)
(684, 241)
(684, 683)
(685, 686)
(685, 687)
(688, 686)
(688, 687)
(689, 686)
(689, 687)
(690, 691)
(690, 692)
(690, 693)
(690, 694)
(695, 691)
(695, 692)
(695, 693)
(695, 694)
(696, 697)
(696, 698)
(696, 699)
(696, 700)
(135, 697)
(135, 698)
(135, 699)
(135, 700)
(701, 8)
(702, 8)
(506, 50

(1621, 660)
(1621, 661)
(427, 660)
(427, 661)
(1867, 660)
(1867, 661)
(27, 660)
(27, 661)
(1868, 780)
(1359, 780)
(132, 132)
(132, 1869)
(132, 1870)
(1869, 132)
(1869, 1869)
(1869, 1870)
(1870, 132)
(1870, 1869)
(1870, 1870)
(8, 1871)
(8, 1872)
(8, 1873)
(8, 1874)
(141, 1875)
(141, 1876)
(141, 27)
(141, 1877)
(141, 1878)
(142, 1875)
(142, 1876)
(142, 27)
(142, 1877)
(142, 1878)
(800, 1875)
(800, 1876)
(800, 27)
(800, 1877)
(800, 1878)
(203, 1875)
(203, 1876)
(203, 27)
(203, 1877)
(203, 1878)
(181, 253)
(181, 254)
(182, 253)
(182, 254)
(20, 1879)
(20, 1880)
(20, 1881)
(20, 1882)
(1883, 1879)
(1883, 1880)
(1883, 1881)
(1883, 1882)
(1884, 1885)
(1886, 1885)
(1887, 1885)
(1888, 1885)
(8, 1889)
(8, 1890)
(1891, 8)
(1892, 1893)
(1892, 1894)
(1892, 1895)
(191, 838)
(192, 838)
(193, 838)
(194, 838)
(1896, 418)
(1896, 1652)
(1896, 865)
(1896, 1897)
(1898, 418)
(1898, 1652)
(1898, 865)
(1898, 1897)
(268, 418)
(268, 1652)
(268, 865)
(268, 1897)
(1899, 1900)
(1901, 1902)
(1901, 1903)
(1901, 1904)


(1932, 650)
(2663, 2664)
(2665, 2664)
(2666, 8)
(2667, 8)
(2668, 2669)
(2668, 2670)
(969, 2671)
(969, 2672)
(971, 2671)
(971, 2672)
(972, 2671)
(972, 2672)
(973, 2671)
(973, 2672)
(8, 779)
(8, 241)
(8, 242)
(2673, 349)
(2673, 767)
(2673, 802)
(2673, 512)
(512, 349)
(512, 767)
(512, 802)
(512, 512)
(1875, 8)
(2674, 2675)
(2674, 1063)
(2676, 2675)
(2676, 1063)
(8, 8)
(2677, 969)
(2677, 971)
(2677, 2678)
(2677, 634)
(2679, 1564)
(942, 1564)
(2680, 2681)
(2680, 1207)
(2682, 2681)
(2682, 1207)
(2683, 2681)
(2683, 1207)
(2684, 2685)
(2684, 2686)
(2684, 2687)
(2688, 2685)
(2688, 2686)
(2688, 2687)
(8, 8)
(2689, 2690)
(2689, 2691)
(815, 2690)
(815, 2691)
(300, 8)
(243, 8)
(2692, 2693)
(2692, 2694)
(2692, 2224)
(990, 2693)
(990, 2694)
(990, 2224)
(8, 1795)
(2695, 8)
(104, 8)
(2696, 2697)
(2696, 2698)
(2696, 2699)
(1216, 2697)
(1216, 2698)
(1216, 2699)
(2700, 2701)
(2702, 2701)
(2703, 2701)
(2704, 2701)
(27, 2701)
(2705, 2701)
(1871, 2706)
(2707, 2706)
(2708, 2709)
(2708, 871)
(2708, 27)
(2708, 

(803, 3361)
(803, 3362)
(804, 3361)
(804, 3362)
(3363, 3364)
(3363, 3365)
(3366, 3364)
(3366, 3365)
(3367, 838)
(3368, 838)
(3369, 2608)
(3369, 330)
(3369, 147)
(187, 2608)
(187, 330)
(187, 147)
(188, 2608)
(188, 330)
(188, 147)
(3370, 2243)
(3371, 2243)
(8, 3372)
(8, 3373)
(8, 3374)
(8, 3375)
(3376, 8)
(719, 8)
(720, 8)
(3180, 8)
(3377, 8)
(3378, 8)
(3379, 8)
(534, 98)
(3011, 3380)
(3011, 3381)
(3011, 3382)
(1696, 3380)
(1696, 3381)
(1696, 3382)
(2690, 3383)
(2691, 3383)
(3384, 1920)
(3384, 3385)
(3384, 27)
(3384, 3386)
(3384, 3387)
(3388, 1920)
(3388, 3385)
(3388, 27)
(3388, 3386)
(3388, 3387)
(982, 3389)
(982, 48)
(3390, 1703)
(3390, 409)
(3391, 1703)
(3391, 409)
(2309, 1703)
(2309, 409)
(3392, 1703)
(3392, 409)
(3393, 1065)
(3394, 3395)
(3394, 1959)
(3396, 3395)
(3396, 1959)
(3397, 3395)
(3397, 1959)
(3398, 3395)
(3398, 1959)
(3399, 8)
(3400, 8)
(27, 8)
(542, 8)
(1293, 3401)
(1294, 3401)
(2954, 2954)
(2954, 3402)
(2954, 3403)
(3402, 2954)
(3402, 3402)
(3402, 3403)
(3403, 2954)
(340

...

(1703, 2635)
(1703, 797)
(409, 3293)
(409, 2635)
(409, 797)
(8, 14780)
(8, 14781)
(8, 14782)
(8828, 3937)
(8828, 238)
(8830, 3937)
(8830, 238)
(4736, 1132)
(4736, 2311)
(4736, 1351)
(4736, 62)
(4736, 4252)
(4736, 27)
(5287, 16991)
(5287, 4701)
(10417, 16991)
(10417, 4701)
(4701, 16991)
(4701, 4701)
(8, 2307)
(8, 13429)
(8, 2308)
(8, 2309)
(4835, 5047)
(2730, 5047)
(7921, 4769)
(7921, 6070)
(4722, 191)
(4722, 192)
(4722, 193)
(4722, 194)
(9145, 191)
(9145, 192)
(9145, 193)
(9145, 194)
(657, 10311)
(657, 10312)
(4415, 10311)
(4415, 10312)
(1910, 8)
(3670, 7679)
(16992, 7679)
(8, 656)
(8, 774)
(8, 27)
(8, 662)
(16993, 8)
(16994, 8)
(1579, 2833)
(1579, 989)
(1579, 990)
(1963, 534)
(1963, 4483)
(1963, 4484)
(1967, 534)
(1967, 4483)
(1967, 4484)
(1966, 534)
(1966, 4483)
(1966, 4484)
(734, 9198)
(735, 9198)
(11044, 9198)
(15177, 9198)
(1822, 8)
(1823, 8)
(1824, 8)
(1825, 8)
(1826, 8)
(4608, 164)
(4608, 168)
(4609, 164)
(4609, 168)
(4610, 164)
(4610, 168)
(8, 2011)
(8, 2012)
(8, 206)
(8, 207)


(8, 3872)
(8, 4071)
(8, 1707)
(8, 1708)
(11, 13984)
(11, 13985)
(11, 6349)
(11, 13986)
(1953, 13984)
(1953, 13985)
(1953, 6349)
(1953, 13986)
(13317, 13984)
(13317, 13985)
(13317, 6349)
(13317, 13986)
(4045, 4045)
(4045, 4048)
(4045, 4047)
(4047, 4045)
(4047, 4048)
(4047, 4047)
(2307, 2037)
(2307, 4806)
(2309, 2037)
(2309, 4806)
(3392, 2037)
(3392, 4806)
(206, 562)
(206, 899)
(207, 562)
(207, 899)
(1632, 1534)
(1671, 8)
(255, 8)
(945, 14308)
(945, 14309)
(3681, 14308)
(3681, 14309)
(947, 14308)
(947, 14309)
(498, 8)
(499, 8)
(500, 8)
(13976, 9510)
(13976, 1887)
(13976, 10349)
(10880, 9510)
(10880, 1887)
(10880, 10349)
(206, 8415)
(206, 8416)
(5711, 4051)
(7374, 4051)
(2071, 12007)
(2071, 12564)
(5576, 16724)
(5576, 16725)
(5576, 16726)
(5576, 16727)
(10621, 16724)
(10621, 16725)
(10621, 16726)
(10621, 16727)
(10622, 16724)
(10622, 16725)
(10622, 16726)
(10622, 16727)
(8, 277)
(8, 7455)
(8, 6770)
(4293, 3352)
(4293, 355)
(17044, 3352)
(17044, 355)
(17045, 8)
(4740, 8)
(8, 8105)
(8, 62)


(584, 393)
(584, 5416)
(584, 3911)
(16364, 3961)
(16364, 393)
(16364, 5416)
(16364, 3911)
(15580, 4368)
(15580, 3079)
(15580, 973)
(5540, 4368)
(5540, 3079)
(5540, 973)
(3459, 13179)
(3459, 9991)
(3459, 13180)
(1306, 15109)
(875, 1630)
(875, 1631)
(875, 376)
(8, 471)
(8, 2266)
(8, 650)
(563, 3384)
(563, 566)
(563, 567)
(566, 3384)
(566, 566)
(566, 567)
(567, 3384)
(567, 566)
(567, 567)
(8848, 8)
(8, 14838)
(8, 14839)
(8, 2655)
(8, 27)
(8, 27)
(14265, 8625)
(14265, 827)
(14265, 7428)
(6904, 8625)
(6904, 827)
(6904, 7428)
(15647, 8)
(80, 3610)
(80, 758)
(669, 13221)
(5975, 13221)
(3154, 1483)
(3154, 58)
(2001, 1483)
(2001, 58)
(1711, 8)
(1712, 8)
(1686, 8)
(514, 5077)
(1803, 5077)
(6928, 5563)
(6928, 4500)
(6928, 3907)
(6928, 5568)
(6928, 27)
(6929, 5563)
(6929, 4500)
(6929, 3907)
(6929, 5568)
(6929, 27)
(2989, 9659)
(2989, 6315)
(997, 9659)
(997, 6315)
(2019, 2883)
(2019, 6149)
(2020, 2883)
(2020, 6149)
(2160, 1061)
(2160, 1064)
(2160, 1063)
(150, 8)
(6163, 8)
(409, 8)
(2928, 8)
(13560,

(1338, 27)
(16031, 6321)
(16031, 6322)
(16031, 6323)
(16031, 27)
(4003, 1713)
(4004, 1713)
(4005, 1713)
(534, 11521)
(534, 11522)
(13109, 3147)
(13894, 3147)
(2592, 8)
(2593, 8)
(6047, 35)
(6047, 1212)
(796, 300)
(796, 69)
(238, 300)
(238, 69)
(1509, 6150)
(1509, 6152)
(3358, 6150)
(3358, 6152)
(2497, 8)
(3964, 2628)
(3964, 2632)
(7938, 2946)
(7938, 8992)
(7938, 27)
(8, 6300)
(2891, 8)
(588, 8787)
(588, 14147)
(588, 14148)
(3149, 8787)
(3149, 14147)
(3149, 14148)
(3634, 8787)
(3634, 14147)
(3634, 14148)
(268, 8787)
(268, 14147)
(268, 14148)
(7398, 4820)
(7398, 12138)
(7398, 12139)
(85, 4820)
(85, 12138)
(85, 12139)
(8838, 4820)
(8838, 12138)
(8838, 12139)
(10631, 11846)
(10631, 757)
(1248, 11846)
(1248, 757)
(27, 11846)
(27, 757)
(1569, 11846)
(1569, 757)
(4835, 660)
(4835, 661)
(4835, 27)
(4835, 662)
(662, 660)
(662, 661)
(662, 27)
(662, 662)
(3384, 8)
(566, 8)
(567, 8)
(8, 8)
(17151, 1768)
(17151, 1769)
(17151, 1770)
(17151, 1771)
(612, 1768)
(612, 1769)
(612, 1770)
(612, 1771)
(1355

(3657, 8157)
(3657, 2987)
(1509, 2621)
(1509, 5134)
(3358, 2621)
(3358, 5134)
(860, 838)
(860, 3205)
(865, 838)
(865, 3205)
(350, 838)
(350, 3205)
(866, 838)
(866, 3205)
(12885, 9406)
(12885, 9159)
(12885, 3830)
(12886, 9406)
(12886, 9159)
(12886, 3830)
(109, 1735)
(109, 8569)
(109, 2231)
(109, 2647)
(109, 14144)
(2123, 1735)
(2123, 8569)
(2123, 2231)
(2123, 2647)
(2123, 14144)
(4794, 1735)
(4794, 8569)
(4794, 2231)
(4794, 2647)
(4794, 14144)
(8, 618)
(8, 3577)
(8, 27)
(8, 512)
(726, 3433)
(726, 1129)
(726, 8447)
(8, 905)
(8, 906)
(8, 229)
(17198, 6171)
(17198, 6172)
(118, 6171)
(118, 6172)
(4947, 7512)
(4947, 143)
(4947, 203)
(6901, 7512)
(6901, 143)
(6901, 203)
(238, 7512)
(238, 143)
(238, 203)
(967, 948)
(731, 291)
(731, 58)
(14990, 291)
(14990, 58)
(12274, 291)
(12274, 58)
(5448, 8)
(114, 563)
(114, 566)
(114, 567)
(2115, 7839)
(2115, 7550)
(2115, 4132)
(2115, 7840)
(17199, 1308)
(1797, 8)
(17200, 8)
(17201, 12958)
(17201, 17202)
(17201, 17203)
(17204, 12958)
(17204, 17202)
(17204,

In [11]:
# we will use three basic features:

# number of overlapping words in title
overlap_title = []

# number of overlapping words in abstract
overlap_abstract = []

# temporal distance between the papers
temp_diff = []

# number of common authors
comm_auth = []

# distance between authors
distance_auth = []

# abstract diff
abstract_diff = []

#authors graph 
#g_authors = build_authors_graph(training_set_reduced)

counter = 0
for i in range(len(training_set_reduced)):
    
    
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    #index_source = IDs.index(source)
    #index_target = IDs.index(target)
    try:
        source_info = [element for element in node_info if element[0]==source][0]
    except:
        source_info = [-1,-1,"","","",""] #[-1,-1,"nan","nan","nan","nan"] #["nan"]*6
    
    try:
        target_info = [element for element in node_info if element[0]==target][0]
    except:
        target_info = [-1,-1,"","","",""] #["nan"]*6
    
	# convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
	# remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    abstract_distance = average_sentences_distance(model, source_info[5].split(' '), target_info[5].split(' ') )
    
    overlap_title.append(len(set(source_title).intersection(set(target_title))))
    overlap_abstract.append(len(set(source_info[5].lower().split(" ")).intersection(set(target_info[5].lower().split(" ")))))
    temp_diff.append(int(source_info[1]) - int(target_info[1]))
    comm_auth.append(len(set(source_auth).intersection(set(target_auth))))
    #distance_auth.append(authors_distance(g_authors, source_auth, target_auth))
    abstract_diff.append(abstract_distance)
   
    counter += 1
    if counter % 1000 == True:
        print(counter, "training examples processsed")

1 training examples processsed
1001 training examples processsed
2001 training examples processsed
3001 training examples processsed
4001 training examples processsed
5001 training examples processsed
6001 training examples processsed
7001 training examples processsed
8001 training examples processsed
9001 training examples processsed
10001 training examples processsed
11001 training examples processsed
12001 training examples processsed
13001 training examples processsed
14001 training examples processsed
15001 training examples processsed
16001 training examples processsed
17001 training examples processsed
18001 training examples processsed
19001 training examples processsed
20001 training examples processsed
21001 training examples processsed
22001 training examples processsed
23001 training examples processsed
24001 training examples processsed
25001 training examples processsed
26001 training examples processsed
27001 training examples processsed
28001 training examples processse

In [54]:
np.array(abstract_diff).T

(30776,)

In [12]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
training_features = np.array([overlap_title, temp_diff, comm_auth, overlap_abstract]+[c for c in np.array(abstract_diff).T.tolist()]).T

# scale
training_features = preprocessing.scale(training_features)

# convert labels into integers then into column array
labels = [int(element[2]) for element in training_set_reduced]
labels = list(labels)
labels_array = np.array(labels)





In [56]:
training_features.shape

(30776, 629)

In [13]:


#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100], 'gamma': [0.001, 0.0001]}
#svc = svm.SVC()
#clf = GridSearchCV(svc, parameters, n_jobs=4, verbose=10)
clf_NB = GaussianNB()
clf_NB.fit(training_features, labels_array)

#sorted(clf.cv_results_.keys())

# initialize basic SVM
#classifier = svm.LinearSVC()

# train
#classifier.fit(training_features, labels_array)

GaussianNB(priors=None)

In [ ]:
svc = svm.SVC()
svc.fit(training_features, labels_array)

In [ ]:
c_range =np.random.uniform(10,100,10).tolist() 

#parameters = {'kernel':('rbf'), 'C':[1,10]}
parameters = { 'C':c_range}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, n_jobs=4, verbose=10)
clf.fit(training_features, labels_array)

sorted(clf.cv_results_.keys())

In [16]:
classifier = clf_NB
#classifier.fit(training_features, labels_array)

In [14]:
# test
# we need to compute the features for the testing set

overlap_title_test = []
overlap_abstract_test = []
temp_diff_test = []
comm_auth_test = []
abstract_diff_test = []
counter = 0
for i in range(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    #
    #index_source = IDs.index(source)
    #index_target = IDs.index(target)
    #
    #source = training_set_reduced[i][0]
    #target = training_set_reduced[i][1]
    #
    #index_source = IDs.index(source)
    #index_target = IDs.index(target)
    
    #source_info = [element for element in node_info if element[0]==source][0]
    #target_info = [element for element in node_info if element[0]==target][0]
    
    try:
        source_info = [element for element in node_info if element[0]==source][0]
    except:
        source_info = [-1,-1,"","","",""]#["nan"]*6
    
    try:
        target_info = [element for element in node_info if element[0]==target][0]
    except:
        target_info = [-1,-1,"","","",""] #["nan"]*6
    
    source_title = source_info[2].lower().split(" ")
    source_title = [token for token in source_title if token not in stpwds] 
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    abstract_distance = average_sentences_distance(model, source_info[5].split(' '), target_info[5].split(' ') )
    
    overlap_title_test.append(len(set(source_title).intersection(set(target_title))))
    overlap_abstract_test.append(len(set(source_info[5].lower().split(" ")).intersection(set(target_info[5].lower().split(" ")))))
    temp_diff_test.append(int(source_info[1]) - int(target_info[1]))
    comm_auth_test.append(len(set(source_auth).intersection(set(target_auth))))
    abstract_diff_test.append(abstract_distance)
   
    counter += 1
    if counter % 1000 == True:
        print (counter, "testing examples processsed")
        


1 testing examples processsed
1001 testing examples processsed
2001 testing examples processsed
3001 testing examples processsed
4001 testing examples processsed
5001 testing examples processsed
6001 testing examples processsed
7001 testing examples processsed
8001 testing examples processsed
9001 testing examples processsed
10001 testing examples processsed
11001 testing examples processsed
12001 testing examples processsed
13001 testing examples processsed
14001 testing examples processsed
15001 testing examples processsed
16001 testing examples processsed
17001 testing examples processsed
18001 testing examples processsed
19001 testing examples processsed
20001 testing examples processsed
21001 testing examples processsed
22001 testing examples processsed
23001 testing examples processsed
24001 testing examples processsed
25001 testing examples processsed
26001 testing examples processsed
27001 testing examples processsed
28001 testing examples processsed
29001 testing examples proc

In [17]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
testing_features = np.array([overlap_title_test,temp_diff_test,comm_auth_test, overlap_abstract_test]+[c for c in np.array(abstract_diff_test).T.tolist()]).T
#training_features = np.array([overlap_title, temp_diff, comm_auth, overlap_abstract]).T

# scale
testing_features = preprocessing.scale(testing_features)

# issue predictions
predictions_SVM = list(classifier.predict(testing_features))

# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
predictions_SVM = zip(range(len(testing_set)), predictions_SVM)

In [18]:
with open("improved_predictions.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    for row in predictions_SVM:
        csv_out.writerow(row)
        
#with open("random_predictions.csv","w") as pred:
#    csv_out = csv.writer(pred)
#    for row in random_predictions:
#        csv_out.writerow(row)